##   Introduction

With this first project we are going to practice a basic EDA using a dataset from the bootcamp ___TripleTen DataScienti Course___. We are going to compare the musical preferences between two cities Springfield and Shelbyville, and confirm the following hypothesis:

- The activity of users differs depending on the day of the week and the city

